In [58]:
class Maze:
    def __init__(self, maze_matrix):
        self.matrix = maze_matrix
        self.row_size = len(self.matrix)
        self.col_size = len(self.matrix[0])
        self.solution = [[' - ' for _ in range(self.col_size)] for _ in range(self.row_size)]
        self.visited =  [[False for _ in range(self.col_size)] for _ in range(self.row_size)]
        self.moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Down, Up, Right, Left

    def is_valid(self, row, col):
        # check the board boundary
        if row < 0 or row >= self.row_size or col < 0 or col >= self.col_size:
            return False

        # check obstacle (wall)
        if self.matrix[row][col] == 0:
            return False
        
        # not valid if it is already visited (already included that cell in the solution)
        if self.visited[row][col]:
            return False
        
        return True

    def is_destination(self, x, y):
        if x == self.row_size - 1 and y == self.col_size - 1:
            return True
        return False
    
    def show_result(self):
        for x in range(self.row_size):
            for y in range(self.col_size):
                print(self.solution[x][y], end=' ')
            print('\n')        


class MazeDFS(Maze):
    def __init__(self, maze_matrix):
        super().__init__(maze_matrix)

    def search(self):
        self.visited[0][0] = True        
        self.solution[0][0] = ' S '  # start at (0, 0)

        if self.solve(0, 0):
            self.show_result()
        else:
            print('There is no feasible solution...')

    def solve(self, x, y):
        if self.is_destination(x, y):
            return True

        for move in self.moves:

            next_x = x + move[0]
            next_y = y + move[1]

            if self.is_valid(next_x, next_y):
                self.visited[next_x][next_y] = True
                self.solution[next_x][next_y] = ' S '

                if self.solve(next_x, next_y):  # DFS
                    return True
                
                # Backtracking!
                # self.visited[next_x][next_y] = False 
                self.solution[next_x][next_y] = ' B '

        return False

In [59]:
from collections import deque

class MazeBFS(Maze):
    def __init__(self, maze_matrix):
        super().__init__(maze_matrix)
        self.edgeTo = [[(-1, -1) for _ in range(self.col_size)] for _ in range(self.row_size)]
        self.min_distance = float('inf')

    def search(self):
        self.__search(0, 0, self.row_size-1, self.col_size-1)
        
    def __search(self, i, j, destination_x, destination_y):

        self.visited[i][j] = True
        # the queue is implemented with a doubly linked list - O(1)
        queue = deque()
        # i is the x coordinate
        # j is the y coordinate
        # why 0? of course because in the first iteration the min_distance is 0
        queue.append((i, j, 0))

        while queue:

            # we take the first item we have inserted
            (i, j, dist) = queue.popleft()

            # if we have reached the destination - we break out of the while loop becase
            # we have found the destination !!!
            if i == destination_x and j == destination_y:
                self.min_distance = dist
                break

            # we are at the location (i,j) we have to make a given move
            # L, U, R, D
            for move in self.moves:
                # we calculate the position ofter the move
                next_x = i + move[0]
                next_y = j + move[1]

                # is it possible to make the move to cell with coordinates (next_x, next_y)?
                if self.is_valid(next_x, next_y):
                    # we make the given move (BFS)
                    self.visited[next_x][next_y] = True
                    # we append the move to the queue
                    queue.append((next_x, next_y, dist + 1))
                    # Add path
                    self.edgeTo[next_x][next_y] = (i, j)
        
        if self.min_distance != float('inf'):
            print("The shortest path from source to destination: ", self.min_distance)
            self.mark_shortest_path()
            self.show_result()
        else:
            print("No feasible solution - the destination can not be reached!")

    def mark_shortest_path(self):
        path = []  # stack
        v = (self.row_size-1, self.col_size-1)
        path.append(v)
        while v != (0, 0):
            s = self.edgeTo[v[0]][v[1]]
            path.append(s)
            v = s      
        for (x, y) in path:
            self.solution[x][y] = ' S '


In [60]:
# 1: valid cells 0: walls or obstacles
maze = [[1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 0],
        [1, 0, 1, 1, 1, 1]]
maze_dfs = MazeDFS(maze)
maze_dfs.search()
print()
maze_bfs = MazeBFS(maze)
maze_bfs.search()

 S   S   S   S   S   S  

 B   -   -   -   S   S  

 B   B   S   S   S   B  

 B   -   S   -   -   -  

 B   -   S   S   S   S  


The shortest path from source to destination:  9
 S   -   -   -   -   -  

 S   -   -   -   -   -  

 S   S   S   -   -   -  

 -   -   S   -   -   -  

 -   -   S   S   S   S  

